## Part 4 - Deploy Model

**Connect to Azure ML workspace and get deployed model**

In [1]:
from azureml.core import Workspace
from azureml.core import Model

workspace = Workspace.from_config()
model = Model(workspace, name='cifar-classifier')

**Define score file and environment (inference configuration)**

In [3]:
%pycat deploy/score.py

In [4]:
%pycat deploy/env.yml

In [2]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory='deploy',
                                   runtime='python',
                                   entry_script='score.py',
                                   conda_file='env.yml')

**Deploy as local web service**

In [3]:
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.webservice import LocalWebservice

local_service_name = 'local-cifar-classifier'
local_config = LocalWebservice.deploy_configuration(port=3000)

try:
    local_service = Webservice(workspace, name=local_service_name)
    if local_service:
        local_service.delete()
except WebserviceException as e:
    print()

local_service = Model.deploy(workspace, local_service_name, [model], inference_config, local_config)
local_service.wait_for_deployment()


Generating Docker build context.
2020/02/19 07:54:49 Downloading source code...
2020/02/19 07:54:50 Finished downloading source code
2020/02/19 07:54:51 Creating Docker network: acb_default_network, driver: 'bridge'
2020/02/19 07:54:51 Successfully set up Docker network: acb_default_network
2020/02/19 07:54:51 Setting up Docker configuration...
2020/02/19 07:54:52 Successfully set up Docker configuration
2020/02/19 07:54:52 Logging in to registry: uaeworkshop5e50fcc4.azurecr.io
2020/02/19 07:54:53 Successfully logged into uaeworkshop5e50fcc4.azurecr.io
2020/02/19 07:54:53 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/02/19 07:54:53 Scanning for dependencies...
2020/02/19 07:54:54 Successfully scanned dependencies
2020/02/19 07:54:54 Launching container with name: acb_step_0
Sending build context to Docker daemon  60.93kB
Step 1/15 : FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04@sha256:a1b514f3ba884b9a7695


#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_01e6b924c5f1d28e8804005fb447330c
#
# To deactivate an active environment, use:
# > source deactivate
#


Removing intermediate container b63ea1178313
 ---> b417ef627971
Step 9/15 : ENV PATH /azureml-envs/azureml_01e6b924c5f1d28e8804005fb447330c/bin:$PATH
 ---> Running in 0ffa05adcc21
Removing intermediate container 0ffa05adcc21
 ---> 7f0aaecf41e6
Step 10/15 : ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/azureml_01e6b924c5f1d28e8804005fb447330c
 ---> Running in 7c706a8712d7
Removing intermediate container 7c706a8712d7
 ---> 8ddedf14443e
Step 11/15 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_01e6b924c5f1d28e8804005fb447330c/lib:$LD_LIBRARY_PATH
 ---> Running in efc446a209b3
Removing intermediate container efc446a209b3
 ---> 456ef607a693
Step 12/15 : COPY azureml-environment-setup/spark_cache.py azureml-environment-setup/log4j.properties /azureml-environment-setup/
 ---> b28184ee1945
Step 13/15 : RUN if 

**Test local service**

In [18]:
from io import BytesIO
import urllib.request
import base64
import io
import requests 
import json
import argparse
from PIL import Image

def imgToBase64(img):
    '''Convert pillow image to base64-encoded image'''
    imgio = BytesIO()
    img.save(imgio, 'JPEG')
    img_str = base64.b64encode(imgio.getvalue())
    return img_str.decode('utf-8')

def test_service(image_url, scoring_url):
    # Download image and convert to base 64
    with urllib.request.urlopen(image_url) as url:
        test_img = io.BytesIO(url.read())

    base64Img = imgToBase64(Image.open(test_img))
    
    # Get prediciton through endpoint
    input_data = '{\"data\": \"'+ base64Img +'\"}'
    headers = {'Content-Type':'application/json'}
    response = requests.post(scoring_url, input_data, headers=headers)
    return json.loads(response.text)

In [16]:
scoring_url = local_service.scoring_uri
image_url = 'https://content.presspage.com/uploads/2431/1920_cairoa380new-135207.jpg?10000'

prediction = test_service(image_url, scoring_url)
print(prediction)

{"label": "airplane", "probability": "0.9789974"}


**Deploy as AKS service**

In [25]:
from azureml.core.webservice import AksWebservice
from azureml.core.compute import AksCompute

aks_compute_target = AksCompute(workspace, 'aks-cluster')

aks_service_name = 'aks-cifar-classifier'
aks_config = AksWebservice.deploy_configuration(cpu_cores=2, memory_gb=4, auth_enabled=False)

try:
    aks_service = Webservice(workspace, name=aks_service_name)
    if aks_service:
        aks_service.delete()
except WebserviceException as e:
    print()

aks_service = Model.deploy(workspace, aks_service_name, [model], inference_config, aks_config, aks_compute_target)
aks_service.wait_for_deployment()

AKS service creation operation finished, operation "Succeeded"


**Test AKS service**

In [26]:
scoring_url = aks_service.scoring_uri
image_url = 'https://content.presspage.com/uploads/2431/1920_cairoa380new-135207.jpg?10000'

prediction = test_service(image_url, scoring_url)
print(prediction)

<Response [200]>
{"label": "airplane", "probability": "0.9789974"}
